In [2]:
import cv2
import numpy as np
import os
import pandas as pd

from sklearn.model_selection import train_test_split
from skimage import exposure

train_images_files     = 'train_ims'
train_label_file = 'train.csv'
# test_images_files      = 'test_ims'
# test_label_file      = 'test.csv'

def load_data(dataset_path, csv_path):
    '''
    Load the image data and labels.
    '''
    
    df = pd.read_csv(csv_path)
    images = []
    labels = []
    
    for index, row in df.iterrows():
        image_name = row['im_name']
        label      = row['label']
        
        image_path  = os.path.join(dataset_path, image_name)
        image       = cv2.imread(image_path)
        images.append(image)
        labels.append(label)
        
    return np.array(images), np.array(labels)



xs, ys = load_data(train_images_files, train_label_file)
x_train, x_test, y_train, y_test = train_test_split(
    xs, 
    ys, 
    test_size=0.2, 
    random_state=42,
    stratify=ys 
)

In [3]:
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(40000, 32, 32, 3) (40000,) (10000, 32, 32, 3) (10000,)


In [4]:
import numpy as np

def flatten_images(images):
    """
    Every pixel has 3 rgb values.
    Treat every rgb value as a single attribute. Therefore, for every image, there will be 32 x 32 x 3 attributes
    """
    flattened_images = images.reshape(len(images), -1)
    return flattened_images

x_train = flatten_images(x_train)
x_test = flatten_images(x_test)

print(x_train.shape)

(40000, 3072)


In [5]:
from sklearn.decomposition import PCA

def pca(xs, n_components):
    pca = PCA(n_components=n_components)
    xs_reduced = pca.fit_transform(xs)
    
    return xs_reduced

pca_dimensions = [400, 200, 100, 50]

# Apply PCA for each dimension and visualize the results
# for dim in pca_dimensions:
#     xs_reconstructed = pca_and_reproject(x_train, n_components=dim)
x_train = pca(x_train, 100)
x_test = pca(x_test, 100)
print(x_train.shape)


(40000, 100)


In [ ]:
from sklearn.preprocessing import StandardScaler

def scale(xs):
    stdsc = StandardScaler()
    return stdsc.fit_transform(xs)

# Scale to speed up convergence
x_train = scale(x_train)
x_train, y_train = x_train[:10000], y_train[:10000]
print(x_train.shape, y_train.shape)

(10000, 100) (10000,)


NameError: name 'device_lib' is not defined

In [7]:
# SVM
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

svm_model = SVC(kernel='linear', C=1, random_state=1)
svm_model.fit(x_train, y_train)

y_pred = svm_model.predict(x_test)

# Step 6: Evaluate the Model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.201

Classification Report:
               precision    recall  f1-score   support

           0       0.19      0.32      0.24      1002
           1       0.23      0.07      0.11      1002
           2       0.15      0.04      0.06      1008
           3       0.16      0.03      0.05      1001
           4       0.18      0.12      0.15       999
           5       0.19      0.13      0.15       999
           6       0.16      0.46      0.24       991
           7       0.17      0.03      0.05      1000
           8       0.25      0.42      0.32      1004
           9       0.26      0.40      0.32       994

    accuracy                           0.20     10000
   macro avg       0.19      0.20      0.17     10000
weighted avg       0.19      0.20      0.17     10000

